In [13]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from scipy.interpolate import PchipInterpolator
from scipy.special import expit


In [46]:
# Purkinje AP is a hybrid between pacemaker AP and atrial AP

# t0 - duration of phase 0
# t1 - duration of phase 1
# t2 - duration of phase 2
# t3 - duration of phase 3
# t4 - duration of phase 4

# t01 - time of shift from phase 0 to phase 1
# t12 - time of shift from phase 1 to phase 2
# t23 - time of shift from phase 2 to phase 3
# t34 - time of shift from phase 3 to phase 4

# V01 - potential at t01 (peak potential)
# V12 - potential at t12 (plateau potential)
# V23 - potential at t23 (late repolarization potential)
# V34 - potential at t34 (true resting membrane potential)

# translation from variables to sliders:

# V40 - Threshold Potential
# V01 - Peak Membrane Potential
# V12 - Plateau Potential
# V23 - Late Repolarization Potential
# V34 - Maximum Diastolic Potential

# t0 - Rapid Depolarization Duration
# t1 - Early Rapid Repolarization Duration
# t2 - Plateau Duration
# t3 - Final Repolarizarion Duration
# t4 - Slow Depolarization Duration

In [47]:
def smooth_transition(x, x0, eps=0.005):
    """
    Sigmoid-based transition function.
    Returns values smoothly varying from 0 to 1 around x0.
    
    eps controls the "width" of the transition region.
    Smaller eps = sharper transition.
    """
    return 1.0 / (1.0 + np.exp(-(x - x0)/eps))

In [48]:
def fast_upstroke(t, V_start, V_end, t01):
    """Steep sigmoid upstroke, numerically stable"""
    k = max(t01, 1e-4) / 12.0
    t_mid = 0.6 * t01
    return V_start + (V_end - V_start) * expit((t - t_mid)/k)

In [61]:
def purkinje_AP(t, 
                V34, V40, V01, V12, V23,
                t01, t12, t23, t34, t40,
                eps):
    """
    Returns one full AP cycle including phase 4 slow depolarization.
    """

    if V34 >= V40:
        V40 = V34 + 1.0

    t_rest = t40 - t34
    t_shift = t - t_rest

    #  Phase 4
    P4 = V34 + (V40 - V34) * np.clip(t / t_rest, 0, 1)

    # Phase 0
    P0 = fast_upstroke(t_shift, V40, V01, t01)

    # Phases 1-3
    knots_t = [t01, t12, t23, t34]
    knots_V = [V01, V12, V23, V34]
    spline = PchipInterpolator(knots_t, knots_V)
    P13 = spline(t_shift)

    # t_shift_1 = t - t_rest - t01 - t12
    # knots_t = [t01, t12, t23]
    # knots_V = [V01, V12, V23]
    # spline = PchipInterpolator(knots_t, knots_V)
    # P12 = spline(t_shift_1)

    # t_shift_2 = t_shift - t23
    # knots_t2 = [t23, t34]
    # knots_V2 = [V23, V34]
    # spline2 = PchipInterpolator(knots_t2, knots_V2)
    # P23 = spline2(t_shift_2)

    # # Smooth transition between phase 4 and phase 0
    # w40 = smooth_transition(t, t40, eps)
    # w23 = smooth_transition(t, t23, eps)
    # w34 = smooth_transition(t, t34, eps)
    # V = (1 - w40)*P4 + w40*((1 - w34)*P0 + w34*((1 - w23)*P12 + w23*P23))

    # Total potential

    # V = np.where((t_shift >= t01) & (t_shift < t34), P13, V)
    # Smooth transition between phase 4 and phase 0 
    w40 = smooth_transition(t, t40, eps) 
    P40 = (1 - w40) * P4 + w40 * P0 
    
    # Total potential 
    V = np.where(t_shift < t01, P40, P13)
    return V

In [62]:
def interactive_purkinje(V34, V40, V01, V12, V23,
                         t0, t1, t2, t3, t4,
                         n_cycles, eps):
    
    t0, t1, t2, t3, t4 = (x / 1000 for x in (t0, t1, t2, t3, t4))

    t01 = t0
    t12 = t01 + t1
    t23 = t12 + t2
    t34 = t23 + t3
    t40 = t34 + t4

    t_cycle = np.linspace(0, t40, 600)
    V_cycle = purkinje_AP(t_cycle,
                          V34, V40, V01, V12, V23,
                          t01, t12, t23, t34, t40,
                          eps)
    
    # Repeat cycles
    t_total = np.linspace(0, n_cycles*t40, n_cycles*len(t_cycle))
    V_total = np.tile(V_cycle, n_cycles)

    plt.figure(figsize=(10,4))
    plt.plot(t_total*1000, V_total, lw=2)
    plt.xlabel("Time (ms)")
    plt.ylabel("Voltage (mV)")
    plt.title(f"Simulated Purkinje Action Potential ({n_cycles} cycles)")
    plt.grid(True)
    plt.ylim(V34-10, V01+20)
    plt.show()

In [63]:
slider_layout = {'width': '600px'} 
slider_style = {'description_width': '250px'}

In [ ]:
interact(interactive_purkinje,
         V34=FloatSlider(value=-90, min=-100, max=-70, step=5, 
                         description="Maximum Diastolic Potential (mV)",
                         style=slider_style, layout=slider_layout),
         V40=FloatSlider(value=-75, min=-85, max=-60, step=5,
                         description="Threshold Potential (mV)",
                         style=slider_style, layout=slider_layout),
         V01=FloatSlider(value=30, min=0, max=50, step=5,
                         description="Peak Membrane Potential (mV)",
                         style=slider_style, layout=slider_layout),
         V12=FloatSlider(value=0, min=-30, max=20, step=5,
                         description="Plateau Potential (mV)",
                         style=slider_style, layout=slider_layout),
         V23=FloatSlider(value=-20, min=-50, max=0, step=5,
                         description="Late Repolarization Potential (mv)",
                         style=slider_style, layout=slider_layout),
         t0=FloatSlider(value=2, min=1, max=10, step=0.5,
                        description="Rapid Depolarization Duration (ms)",
                        style=slider_style, layout=slider_layout),
         t1=FloatSlider(value=5, min=2, max=20, step=1,
                        description="Early Rapid Repolarization Duration (ms)",
                        style=slider_style, layout=slider_layout),
         t2=FloatSlider(value=200, min=50, max=400, step=10,
                        description="Plateau Duration (ms)",
                        style=slider_style, layout=slider_layout),
         t3=FloatSlider(value=100, min=30, max=250, step=10,
                        description="Final Repolarization Duration (ms)",
                        style=slider_style, layout=slider_layout),
         t4=FloatSlider(value=200, min=50, max=300, step=10,
                        description="Slow Depolarization Duration (ms)",
                        style=slider_style, layout=slider_layout),
         n_cycles=IntSlider(value=2, min=1, max=4, step=1,
                            description="Number of AP cycles",
                            style=slider_style, layout=slider_layout),
         eps=FloatSlider(value=0.001, min=0.001, max=0.005, step=0.005,
                         description="Blend width",
                         style=slider_style, layout=slider_layout)
        );

interactive(children=(FloatSlider(value=-90.0, description='Maximum Diastolic Potential (mV)', layout=Layout(w…